In [73]:
import time
import random
import sympy
import math

In [2]:
class rsa_key:
    def check_congruent(self,a,c,m,debug=True):
        if (a % m) != (c % m):
            print("Congruent check not passed.")
            exit(1)
        else:
            if debug:
                print("Congruent check passed.")
            return True

    def __init__(self, bits_modulo=4096, e=2**16+1):
        self.primeP, self.primeQ =  self.getPrimes(e, bits_modulo)
        self.phi = (self.primeP-1)*(self.primeQ-1)
        self.publicExponent = e
        self.privateExponent = int(sympy.gcdex(e, self.phi)[0])
        self.modulus = self.primeP*self.primeQ
        self.privateExponentModulusPhiP = int(self.privateExponent % (self.primeP-1))
        self.privateExponentModulusPhiQ = int(self.privateExponent % (self.primeQ-1))
        self.inverseQModulusP = int(sympy.mod_inverse(self.primeQ, self.primeP))

        self.check_congruent(self.privateExponent * self.publicExponent, 1, self.phi,debug=False)
        self.check_congruent(self.privateExponentModulusPhiP, self.privateExponent, self.primeP-1,debug=False)
        self.check_congruent(self.privateExponentModulusPhiQ, self.privateExponent, self.primeQ-1,debug=False)
        self.check_congruent(self.inverseQModulusP * self.primeQ, 1, self.primeP,debug=False)
    
    def sign(self, message):
        a = pow(message, self.privateExponentModulusPhiP, self.primeP)
        b = pow(message, self.privateExponentModulusPhiQ, self.primeQ)
        qq = self.inverseQModulusP * self.primeQ
        pp = 1 - qq
        return pp*b + qq*a
    
    def sign_slow(self, message):
        return pow(message, self.privateExponent, self.modulus)
    
    def verify(self, message, signature):
        return pow(signature,self.publicExponent,self.modulus) == message


    def getPrimes(self,e, bits_modulo):
        while True:
            p = random.getrandbits(bits_modulo)
            retry = 1
            while not sympy.isprime(p):
                print(retry)
                retry += 1
                p = random.getrandbits(bits_modulo)
            q = random.getrandbits(bits_modulo)
            while not sympy.isprime(q) or p == q:
                print(retry)
                retry += 1
                q = random.getrandbits(bits_modulo)
            if  math.gcd(e,p*q) == 1 and e < (p-1)*(q-1):
                return int(p), int(q)


In [3]:
## DISCLAIMER: Ejecutar el siguiente código puede tardar mucho tiempo. (12 mins aprox)
meanTimes = {}
meanSlowTimes = {}
for bits in [512, 1024, 2048, 4096]:
    print(bits)
    a = rsa_key(bits_modulo=bits)
    times = []
    slow_times = []
    for i in range(100):
        m = random.randint(0, 1000)

        start = time.time()
        a.sign(m)
        end = time.time()

        start_slow = time.time()
        a.sign_slow(m)
        end_slow = time.time()

        times.append(end-start)
        slow_times.append(end_slow-start_slow)

    meanTimes[bits] = sum(times)/len(times)
    meanSlowTimes[bits] = sum(slow_times)/len(slow_times)

512
1024
2048
4096


In [4]:
print(meanTimes)
print(meanSlowTimes)

{512: 0.0023760008811950684, 1024: 0.01295459270477295, 2048: 0.08876752376556396, 4096: 0.6792355799674987}
{512: 0.007959756851196289, 1024: 0.0431899094581604, 2048: 0.4139524507522583, 4096: 3.043053641319275}
